In [ ]:
packages = [
    '\\usepackage[T1]{fontenc}',
    '\\usepackage[utf8]{inputenc}',
    '\\usepackage{lmodern}',
    '\\usepackage{textcomp}',
    '\\usepackage{lastpage}',
    '\\usepackage{graphicx}',
    '\\usepackage{longtable}',
    '\\usepackage[sorting=none]{biblatex}',
    '\\usepackage{booktabs}',
]
title = 'Supplemental Appendix'
bib_filename = 'austcovid'
with open('appendix.tex', 'w') as file:
    file.write('\\documentclass{article}\n')
    for line in packages:
        file.write(f'{line}\n')
    file.write(f'\\addbibresource{{{bib_filename}.bib}}\n')
    file.write(f'\\title{{{title}}}\n')
    file.write('\\begin{document}\n')
    file.write('test text\n')
    file.write('\\printbibliography\n')
    file.write('\\end{document}\n')